In [2]:
import os
from posixpath import join
import paramiko
from scp import SCPClient
import shutil

run_id = 0
jar_name = 'letter-frequency-1.0-SNAPSHOT.jar'
# connect with ssh


c:\Users\Utente\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


First analyze the letter frequencies in books across the eras. As performance is not relevant now, the job will be executed with 1 reducer, using combiner.
We hypotize that the input files are already in the input folder in hdfs, in /user/hadoop/letter_analysis/input/.

In [3]:
""" ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect('10.1.1.124', username='hadoop', password='ubuntu')
root_dir = '/user/hadoop/historical_analysis'

# run the job for each file
for txt_file in os.listdir('../resources/historical_analysis/input'):
    print(f'Processing {txt_file}')
    input_dir = join(root_dir, 'input', txt_file)
    output_dir = join(root_dir, f'output_{run_id}', txt_file.split('.')[0])
    stdin, stdout, stderr = ssh.exec_command(f'/opt/hadoop/bin/hadoop jar {jar_name} it.unipi.cloud.MapReduceApp '
                                             f'{input_dir} {output_dir}/count {output_dir}/freq 1 inmappercombiner')
    print(stderr.read().decode('utf-8'))
    print(stdout.read().decode('utf-8'))

# copy the output to local
stdin, stdout, stderr = ssh.exec_command(f'/opt/hadoop/bin/hadoop fs -copyToLocal /user/hadoop/historical_analysis/output_{run_id} .')
print(stderr.read().decode('utf-8'))

if not os.path.exists(f'../resources/historical_analysis/output_{run_id}'):
    os.mkdir(f'../resources/historical_analysis/output_{run_id}')
    
scp = SCPClient(ssh.get_transport())
scp.get(f'output_{run_id}', '../resources/historical_analysis/', recursive=True)

# remove the output
ssh.exec_command(f'rm -r output_{run_id}')
scp.close()
ssh.close() """

" ssh = paramiko.SSHClient()\nssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())\nssh.connect('10.1.1.124', username='hadoop', password='ubuntu')\nroot_dir = '/user/hadoop/historical_analysis'\n\n# run the job for each file\nfor txt_file in os.listdir('../resources/historical_analysis/input'):\n    print(f'Processing {txt_file}')\n    input_dir = join(root_dir, 'input', txt_file)\n    output_dir = join(root_dir, f'output_{run_id}', txt_file.split('.')[0])\n    stdin, stdout, stderr = ssh.exec_command(f'/opt/hadoop/bin/hadoop jar {jar_name} it.unipi.cloud.MapReduceApp '\n                                             f'{input_dir} {output_dir}/count {output_dir}/freq 1 inmappercombiner')\n    print(stderr.read().decode('utf-8'))\n    print(stdout.read().decode('utf-8'))\n\n# copy the output to local\nstdin, stdout, stderr = ssh.exec_command(f'/opt/hadoop/bin/hadoop fs -copyToLocal /user/hadoop/historical_analysis/output_{run_id} .')\nprint(stderr.read().decode('utf-8'))\n\nif not o

After that we can analyze the output files.

In [4]:
""" import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

run_id = 0

directory = f'../resources/historical_analysis/output_{run_id}'
print(directory)

years = []
for filename in os.listdir(directory):
    year = filename.split('.')[0]
    years.append(year)

df = pd.DataFrame(index=years, columns=[])

for year in years:
    freq_per_year = []
    filepath = f'{directory}/{year}/freq/part-r-00000'
    with open(filepath, 'r') as f:
        for line in f:
            letter, freq = line.strip().split('\t')
            freq = float(freq)
            df.loc[year, letter] = freq
print(df)

df.plot( kind='bar', stacked=True, title='Letter frequency over the years', figsize=(20, 10)) """

" import os\nimport pandas as pd\nimport matplotlib.pyplot as plt\nimport seaborn as sns\n\nrun_id = 0\n\ndirectory = f'../resources/historical_analysis/output_{run_id}'\nprint(directory)\n\nyears = []\nfor filename in os.listdir(directory):\n    year = filename.split('.')[0]\n    years.append(year)\n\ndf = pd.DataFrame(index=years, columns=[])\n\nfor year in years:\n    freq_per_year = []\n    filepath = f'{directory}/{year}/freq/part-r-00000'\n    with open(filepath, 'r') as f:\n        for line in f:\n            letter, freq = line.strip().split('\t')\n            freq = float(freq)\n            df.loc[year, letter] = freq\nprint(df)\n\ndf.plot( kind='bar', stacked=True, title='Letter frequency over the years', figsize=(20, 10)) "

performance analysis

In [5]:
""" import string
import random
# Dimensioni dei file in bytes
sizes = [10 * 1024, 1 * 1024 * 1024, 100 * 1024 * 1024, 1 * 1024 * 1024 * 1024]
chunk_size = 100  # Dimensione del chunk in bytes

# Nomi dei file
file_dir = "../resources/performance_analysis/input"
filenames = [os.path.join(file_dir, "file_10KB.txt"), 
             os.path.join(file_dir, "file_1MB.txt"), 
             os.path.join(file_dir, "file_100MB.txt"), 
             os.path.join(file_dir, "file_1GB.txt")]

# Genera una stringa di lettere casuali
def generate_random_string(length):
    letters = string.ascii_letters
    return ''.join(random.choice(letters) for i in range(length))

for size, filename in zip(sizes, filenames):
    print(f"Generating {filename}...")
    with open(filename, 'w') as f:
        for _ in range(size // chunk_size):
            f.write(generate_random_string(chunk_size))
            f.write('\n')
        remaining = size % chunk_size
        if remaining:
            f.write(generate_random_string(remaining))
    print(f"File {filename} generated successfully.") """

' import string\nimport random\n# Dimensioni dei file in bytes\nsizes = [10 * 1024, 1 * 1024 * 1024, 100 * 1024 * 1024, 1 * 1024 * 1024 * 1024]\nchunk_size = 100  # Dimensione del chunk in bytes\n\n# Nomi dei file\nfile_dir = "../resources/performance_analysis/input"\nfilenames = [os.path.join(file_dir, "file_10KB.txt"), \n             os.path.join(file_dir, "file_1MB.txt"), \n             os.path.join(file_dir, "file_100MB.txt"), \n             os.path.join(file_dir, "file_1GB.txt")]\n\n# Genera una stringa di lettere casuali\ndef generate_random_string(length):\n    letters = string.ascii_letters\n    return \'\'.join(random.choice(letters) for i in range(length))\n\nfor size, filename in zip(sizes, filenames):\n    print(f"Generating {filename}...")\n    with open(filename, \'w\') as f:\n        for _ in range(size // chunk_size):\n            f.write(generate_random_string(chunk_size))\n            f.write(\'\n\')\n        remaining = size % chunk_size\n        if remaining:\n     

In [6]:
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect('10.1.1.124', username='hadoop', password='ubuntu')
root_dir = '/user/hadoop/performance_analysis'
methods = ['combiner', 'inmappercombiner']
n_reducers = 3


# try all the combinations of methods and reducers
for method in methods:
    for i in range(1, n_reducers+1):
        for txt_file in os.listdir('../resources/performance_analysis/input'):
            print(f'Processing {txt_file} with {method} and {i} reducers')
            input_dir = join(root_dir, 'input', txt_file)
            output_dir = join(root_dir, f'output_{run_id}_{method}_{i}', txt_file.split('.')[0])
            ssh.exec_command(f'mkdir -p output_{run_id}_{method}_{i}/{txt_file.split(".")[0]}')

            stdin, stdout, stderr = ssh.exec_command(f'/opt/hadoop/bin/hadoop jar {jar_name} it.unipi.cloud.MapReduceApp '
                                                     f'{input_dir} {output_dir}/count {output_dir}/freq {i} {method} '
                                                     f'> log.txt 2>&1')
            print(stderr.read().decode('utf-8'))
            
            stdin, stdout, stderr = ssh.exec_command(f'/opt/hadoop/bin/hadoop fs -copyToLocal {output_dir} output_{run_id}_{method}_{i}/{txt_file.split(".")[0]}' )
            print(stderr.read().decode('utf-8'))
            stdin, stdout, stderr = ssh.exec_command(f'mv log.txt output_{run_id}_{method}_{i}/{txt_file.split(".")[0]}/log.txt')
            print(stderr.read().decode('utf-8'))
            
        # create the output directory in project resources
        if not os.path.exists(f'../resources/performance_analysis/output_{run_id}_{method}_{i}'):
            os.mkdir(f'../resources/performance_analysis/output_{run_id}_{method}_{i}')

        # move the output to local machine
        scp = SCPClient(ssh.get_transport())
        scp.get(f'output_{run_id}_{method}_{i}', '../resources/performance_analysis/', recursive=True)
        
        # remove the output directory from virtual machine
        stdin, stdout, stderr = ssh.exec_command(f'rm -r output_{run_id}_{method}_{i}')
        print(stderr.read().decode('utf-8'))

scp.close()
ssh.close()

Processing 10KB.txt with combiner and 1 reducers

2024-06-19 23:18:49,107 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2024-06-19 23:18:49,239 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false



Processing 10KB.txt with combiner and 2 reducers
